# 2017 interleaved A/B test of machine learned ranking

Prior to 2016, our search engine used term frequency—inverse document frequency ([tf—idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)) for ranking documents (e.g. articles and other pages on English Wikipedia). After successful A/B testing, we switched to [BM25 scoring algorithm](https://en.wikipedia.org/wiki/Okapi_BM25) which was used production on almost all languages – except a few space-less languages. After that we focused our efforts on information retrieval using [machine-learned ranking](https://en.wikipedia.org/wiki/Learning_to_rank) (MLR). In MLR, a model is trained to predict a document’s relevance from various document-level and query-level features which represent the document.

[MjoLniR](https://gerrit.wikimedia.org/g/search/MjoLniR) – our Python and Spark-based library for handling the backend data processing for Machine Learned Ranking at Wikimedia – uses a click-based [Dynamic Bayesian Network](https://en.wikipedia.org/wiki/Dynamic_Bayesian_network) (Chapelle and Zhang 2009) (implemented via [ClickModels](https://github.com/varepsilon/clickmodels) Python library) to create relevance labels for training data fed into XGBoost.

## Data

This example uses archived data from the 2017 test of our machine-learned ranking functions. The data was collected using the [SearchSatisfaction instrument](https://gerrit.wikimedia.org/r/plugins/gitiles/mediawiki/extensions/WikimediaEvents/+/c7b48d995dbbbe6767b3d3ef452b5aea68ce7a60/modules/ext.wikimediaEvents/searchSatisfaction.js) ([schema](https://gerrit.wikimedia.org/r/plugins/gitiles/schemas/event/secondary/+/30087c7f6910f7ca917ede504f2c8498edb29216/jsonschema/analytics/legacy/searchsatisfaction/current.yaml)). Sessions with 50 or more searches were excluded from the analysis, due to them potentially being automated/bots.

In [1]:
from interleaved import Experiment
import pandas as pd

In [2]:
ltr_test = pd.read_csv('learning_to_rank_2017.csv')
ltr_test = ltr_test.groupby(ltr_test.group_id)

In [3]:
test_1 = ltr_test.get_group("ltr-i-1024")
test_2 = ltr_test.get_group("ltr-i-20")
test_3 = ltr_test.get_group("ltr-i-20-1024")

## BM-25 vs MLR-20

The "MLR-20" model used a rescore window of 20. This means that each shard (of which English Wikipedia has 7) applies the model to the top 20 results. Those 140 results are then collected and sorted to produce the top 20 shown to the user.

In [4]:
experiment_1 = Experiment(
    queries = test_1['search_id'].to_numpy(),
    clicks = test_1['team'].to_numpy()
)
experiment_1.bootstrap(seed=20)
print(experiment_1.summary(rescale=True, ranker_labels=['BM-25', 'MLR-20']))

 In this interleaved search experiment, 4854 searches were used to determine whether the
results from ranker 'BM-25' or 'MLR-20' were preferred by users (based on their clicks to
the results from those rankers interleaved into a single search result set).

 The preference statistic, as defined by Chapelle et al. (2012), was estimated to be -4.0%
with a 95% (bootstrapped) confidence interval of (-6.8%, -1.5%) on [-100%, 100%] scale
with -100% indicating total preference for 'MLR-20', 100% indicating total preference for
'BM-25', and 0% indicating complete lack of preference between the two -- indicating that
the users had preference for ranker 'MLR-20'.


## BM-25 vs MLR-1024

The "MLR-1024" model used a rescore window of 1024. This means that each of the seven shards applies the model to the top 1024 results. Those 7168 results are then collected and sorted to produce the final top 20 (or top 15) shown to the users, since almost no users look at results beyond the first page.

In [5]:
experiment_2 = Experiment(
    queries = test_2['search_id'].to_numpy(),
    clicks = test_2['team'].to_numpy()
)
experiment_2.bootstrap(seed=1024)
print(experiment_2.summary(rescale=True, ranker_labels=['BM-25', 'MLR-1024']))

 In this interleaved search experiment, 4603 searches were used to determine whether the
results from ranker 'BM-25' or 'MLR-1024' were preferred by users (based on their clicks
to the results from those rankers interleaved into a single search result set).

 The preference statistic, as defined by Chapelle et al. (2012), was estimated to be -1.7%
with a 95% (bootstrapped) confidence interval of (-4.4%, 1.4%) on [-100%, 100%] scale with
-100% indicating total preference for 'MLR-1024', 100% indicating total preference for
'BM-25', and 0% indicating complete lack of preference between the two -- indicating that
the users had no preference for either ranker.


## MLR-20 vs MLR-1024

In [6]:
experiment_3 = Experiment(
    queries = test_3['search_id'].to_numpy(),
    clicks = test_3['team'].to_numpy()
)
experiment_3.bootstrap(seed=1004)
print(experiment_3.summary(rescale=True, ranker_labels=['MLR-20', 'MLR-1024']))

 In this interleaved search experiment, 4709 searches were used to determine whether the
results from ranker 'MLR-20' or 'MLR-1024' were preferred by users (based on their clicks
to the results from those rankers interleaved into a single search result set).

 The preference statistic, as defined by Chapelle et al. (2012), was estimated to be -0.1%
with a 95% (bootstrapped) confidence interval of (-3.0%, 2.6%) on [-100%, 100%] scale with
-100% indicating total preference for 'MLR-1024', 100% indicating total preference for
'MLR-20', and 0% indicating complete lack of preference between the two -- indicating that
the users had no preference for either ranker.


------------

## References

Chapelle, O., & Zhang, Y. (2009). *A dynamic bayesian network click model for web search ranking*. New York, New York, USA: ACM.

Chapelle, O., Joachims, T., Radlinski, F., & Yue, Y. (2012). Large-scale validation and analysis of interleaved search evaluation. *ACM Trans. Inf. Syst.*, **30**(1), 6:1–6:41. doi:10.1145/2094072.2094078